In [2]:
import openpyxl
import pickle
from ocelot import *

# CONFIGURATION
excel_file = "injector_lattice_20230303-2.xlsx"
sheet_name = "lattice"

from rfglinacbte1 import lattice_list 
lat = MagneticLattice(lattice_list)

# READ SURVEY DATA
wb = openpyxl.load_workbook(excel_file, data_only=True)
ws = wb[sheet_name]

#  row 12 
headers = [c.value for c in ws[12]]
col_index = {h: i for i, h in enumerate(headers)}
print("Detected headers:", headers)

# Correct column names 
name_col = "element_name"
dx_col = "Δx [mm]"
dz_col = "Δz [mm]"

data = []
for row in ws.iter_rows(min_row=13, values_only=True):
    if not row[col_index[name_col]]:
        continue
    data.append({
        "name": str(row[col_index[name_col]]).strip(),
        "dx": (row[col_index[dx_col]] or 0) * 1e-3,  # mm → m
        "dz": (row[col_index[dz_col]] or 0) * 1e-3,  # mm → m
    })

# Create lookup dictionary
misalign_dict = {d["name"]: d for d in data}

print(f"Loaded {len(data)} misalignment entries")

# BUILD LOOKUP WITH NORMALIZED NAMES
def normalize(name: str):
    """Normalize element names (remove underscores, make uppercase)."""
    return str(name).replace("_", "").upper().strip()

misalign_dict = {normalize(d["name"]): d for d in data}

# APPLY MISALIGNMENTS
applied, missing = [], []

for elem in lat.sequence:
    name = getattr(elem, "id", None)
    if not name:
        continue
    norm_name = normalize(name)
    if norm_name in misalign_dict:
        mis = misalign_dict[norm_name]
        elem.dx = mis["dx"]
        elem.dy = mis["dz"]  # Z = vertical
        applied.append(name)
    else:
        missing.append(name)

print(f"Applied misalignments to {len(applied)} elements")
if missing:
    print(f" {len(missing)} elements not found in Excel list (e.g. {missing[:10]}...)")


# SAVE UPDATED LATTICE
output_file = "lattice_with_misalignments.pkl"
with open(output_file, "wb") as f:
    pickle.dump(lat, f)

print(f"\n Lattice saved as '{output_file}'")


Detected headers: ['beam_source', 'segment_name', 'inst_type', 'd/s', None, 'point_name', 'element_name', 'element_type', 's [m]', 'xd [mm]', 'yd [mm]', 'zd [mm]', 'xs [mm]', 'ys [mm]', 'zs [mm]', "zs' [mm]", 'Δx [mm]', 'Δy [mm]', 'Δz [mm]', "Δz' [mm]", "x' [rad]", "y' [rad]", "z' [rad]", 'drift [mm]', 'element_length [mm]', 'center-to-center_length [mm]', 'effective_length [m]', 'bore_dia [mm]', 'aperture [mm]', 'Remarks', None, None, None]
Loaded 1972 misalignment entries
Applied misalignments to 797 elements
 3582 elements not found in Excel list (e.g. ['PSECTA', 'L0SPA1M', 'LSPA1M', 'L0PDA1M', 'LPDA1M', 'L0PFA1M', 'LPFA1M', 'L0PXA1M', 'LPXA1M', 'L0PYA1M']...)

 Lattice saved as 'lattice_with_misalignments.pkl'


In [3]:

# CHECK MISSING ELEMENT TYPES (INCLUDING CAVITIES)
marker_like_types = {
    "MARK", "MARKER", "MONITOR", "BPM", "DRIFT", "SEXTUPOL", "QUADRUPOLE_MARKER",
    "CAVITY", "RFCAVITY", "RF", "CAV"  # include all cavity naming variations
}

only_markers = True
non_marker_missing = []

for elem in lat.sequence:
    if elem.id in missing:
        etype = elem.__class__.__name__.upper()
        if not any(key in etype for key in marker_like_types):
            non_marker_missing.append((elem.id, etype))
            only_markers = False

if only_markers:
    print("All missing elements are markers, cavities, or similar (no issue).")
else:
    print(f" {len(non_marker_missing)} missing elements are NOT markers or cavities:")
    for name, etype in non_marker_missing[:260]:  # show first 10
        print(f"   {name} ({etype})")



 259 missing elements are NOT markers or cavities:
   PDR001 (QUADRUPOLE)
   PFR001 (QUADRUPOLE)
   PDR002 (QUADRUPOLE)
   PFR002 (QUADRUPOLE)
   BSR01 (SBEND)
   BSR01 (SBEND)
   BSR01 (SBEND)
   BSR02 (SBEND)
   BYR023 (SBEND)
   BSR03 (SBEND)
   BSR04 (SBEND)
   BSR05 (SBEND)
   BSR06 (SBEND)
   BSR06 (SBEND)
   PDR061 (QUADRUPOLE)
   PFR061 (QUADRUPOLE)
   PDR063 (QUADRUPOLE)
   PFR063 (QUADRUPOLE)
   SX123 (SBEND)
   SY123 (SBEND)
   BX135 (QUADRUPOLE)
   BY135 (QUADRUPOLE)
   SSOLB (SOLENOID)
   SSOLE (SOLENOID)
   PD1825 (QUADRUPOLE)
   PF1849 (QUADRUPOLE)
   PF2124 (QUADRUPOLE)
   PF2313 (QUADRUPOLE)
   BC1E (SBEND)
   BC2E (SBEND)
   BC2E (SBEND)
   BC1E (SBEND)
   SX321 (SBEND)
   SY321 (SBEND)
   SX323 (SBEND)
   SY323 (SBEND)
   SX331 (SBEND)
   SY331 (SBEND)
   SX333 (SBEND)
   SY333 (SBEND)
   SX341 (SBEND)
   SY341 (SBEND)
   SX343 (SBEND)
   SY343 (SBEND)
   SX351 (SBEND)
   SY351 (SBEND)
   SX353 (SBEND)
   SY353 (SBEND)
   SX371 (SBEND)
   SY371 (SBEND)
   SX373 (SBEN

In [4]:
import pickle

lat = pickle.load(open("lattice_with_misalignments.pkl", "rb"))

misaligned = []
for e in lat.sequence:
    dx = getattr(e, "dx", 0)
    dy = getattr(e, "dy", 0)
    if abs(dx) > 1e-9 or abs(dy) > 1e-9:  # nonzero
        misaligned.append((e.id, e.__class__.__name__, dx, dy))

print(f"Total misaligned elements: {len(misaligned)}")
for name, etype, dx, dy in misaligned[:10]:
    print(f"{name:15s} {etype:15s} dx={dx:.6e} dy={dy:.6e}")



Total misaligned elements: 797
SPA1M           Drift           dx=-1.367750e-05 dy=-3.915465e-05
PDA1M           Quadrupole      dx=-8.764440e-06 dy=-4.475525e-05
PFA1M           Quadrupole      dx=3.109610e-05 dy=-1.102533e-05
PXA1M           SBend           dx=0.000000e+00 dy=5.580830e-06
PYA1M           SBend           dx=-2.541862e-05 dy=2.169896e-05
SCA1MS          Drift           dx=-1.264082e-05 dy=4.535847e-05
SCA1M           Drift           dx=-6.984679e-07 dy=6.747107e-05
WSA1M           Drift           dx=3.734283e-06 dy=7.567880e-05
GVA20           Drift           dx=2.235184e-05 dy=1.101513e-04
ACA21           Cavity          dx=1.278748e-04 dy=3.055388e-04
